## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pymystem3 import Mystem
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

In [21]:
pd.__version__

'0.25.1'

In [22]:
df = pd.read_csv('/datasets/data.csv')
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


<font color='purple'>Update</font>

In [23]:
df.sample()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
9019,2,-760.461634,34,среднее,1,женат / замужем,0,F,сотрудник,1,88021.177989,профильное образование


#### Описание данных
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

In [24]:
print(f'Количество дубликатов: {df.duplicated().sum()}')

Количество дубликатов: 54


In [25]:
# Посмотрим на типы колонок
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [26]:
# Посмотрим на числовые признаки
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [27]:
# Соответствие NaN days_employed -> NaN total_income
df.days_employed[df.total_income.isna()].isna().sum()

2174

In [28]:
# Стаж больше возраста лет - некорректное значение
(df.days_employed.abs()/365>df.dob_years).sum()

3519

In [29]:
# Посмотрим на категориальные признаки
columns = list(df.select_dtypes(['object']).columns) +['children', 'education_id', 'family_status_id', 'debt']
for column in columns:
    print(' '*10 + column)
    print(df[column].value_counts())
    print('-'*30)

          education
среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64
------------------------------
          family_status
женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64
------------------------------
          gender
F      14236
M       7288
XNA        1
Name: gender, dtype: int64
------------------------------
          income_type
сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безр

In [30]:
# значения family_status и family_status_id взаимозаменяемы
df[['family_status', 'family_status_id']].head(15)

,family_status,family_status_id
0,женат / замужем,0
1,женат / замужем,0
2,женат / замужем,0
3,женат / замужем,0
4,гражданский брак,1
5,гражданский брак,1
6,женат / замужем,0
7,женат / замужем,0
8,гражданский брак,1
9,женат / замужем,0


In [31]:
# значения education и education_id взаимозаменяемы
df[['education', 'education_id']].head(15)

,education,education_id
0,высшее,0
1,среднее,1
2,Среднее,1
3,среднее,1
4,среднее,1
5,высшее,0
6,высшее,0
7,СРЕДНЕЕ,1
8,ВЫСШЕЕ,0
9,среднее,1


### Вывод

В данных 54 дубликата.

В колонках `children`, `days_employed`, `dob_years` и `gender` есть некорректные значения
 - в `children` - значения '20' и '-1', в сумме их около 120 штук, удалим эти значения
 - в `days_employed` есть отрицательные значения и аномально большие значения, исходя из предположения об ошибки парсера данных, уберем знак минус у всех отрицательных значений. Аномально большие значения заменим на среднее
 - в `dob_years` есть значения '0', что может означать пропуск, заменим их на среднее
 - в `gender` есть одно неуказанное значение, его можно удалить
 

В колонках `days_employed` и `total_income` есть пропуски, причем одновременно, - 2174 штуки.

Пропуски в данных можно обработать разными способами: 
 - удалить строчки с пропущенными значениями
 - заполнить каким-либо значением, например, медианой или средним
 - предсказать значения используя данные других колонок, например, с помощью линейной регрессии
 
 
 - в колонке `total_income` на первый взляд выбросов нет, среднее и медиана имеют близкие значения, поэтому заполним пропуски средним 
 - в колонке`days_employed` присутствуют выбросы и некорректные значения, заполним пропуски средним после исправления всех ошибок

Колонки `education_id` и `education`, а так же `family_status_id` и `family_status` взаимозаменяемы, категориальные значения одной колонки переведены в числовые значения другой 

В колонках `education` и `purpose` есть значения отличающиеся регистром или порядком слов.
 - колонку `education` можем удалить, оставив словарь значений для `education_id`
 - в колонке `purpose` выделим леммы и попробуем сгруппировать значения 
 

Все категориальные признаки приведем к типу *category*

### Шаг 2. Предобработка данных

### Обработка пропусков

In [32]:
# Исправим стаж > возраста клиента
df.days_employed = df.days_employed.abs()
df.days_employed[df.days_employed/365>df.dob_years] = np.NaN

In [33]:
# Заполним пропуски
df.days_employed.fillna(df[df.days_employed/365<=df.dob_years].groupby('dob_years')['days_employed'].transform('mean'),
                        inplace=True
                       )
df.total_income.fillna(df.groupby('income_type')['total_income'].transform('median'), inplace=True)

### Вывод

Пропуски в колонке заполнены средними  значениями. Колонка `days_employed` предварительно обработана.
    
- пропуски в колонке `days_employed` заполнены средними в соответствии с возрастом клиента
- пропуски в колонке `total_income` заполнены средними в соответствии с типом занятости клиента 
- колонка `days_employed` предварительно обработана.

P.S. Вероятно, заполнение зарплат и стажа средними не совсем корректно в случае задачи с банков, ведь этих доходов может и не быть у клиента, в следующий раз следует заполнять нулями. </font>

### Удаление  и замена некорректных значений

In [34]:
df.drop(axis=0,index=df[df.gender == 'XNA'].index, inplace=True)
df.drop(axis=0,index=df[df.children == 20].index, inplace=True)
df.drop(axis=0,index=df[df.children == -1].index, inplace=True)
df.dob_years.replace(0, df.dob_years[df.dob_years>0].mean(), inplace=True)

### Вывод

- удалили некорректное значение пола и значения количества детей '20' и '-1'. 
- заменили значение возраста '0'  на среднее значение возраста

### Замена типа данных

In [35]:
int_cols = ['children', 'dob_years', 'education_id', 'family_status_id', 'debt', 'total_income']

df[int_cols] = df[int_cols].astype('int')

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21401 entries, 0 to 21524
Data columns (total 12 columns):
children            21401 non-null int64
days_employed       15735 non-null float64
dob_years           21401 non-null int64
education           21401 non-null object
education_id        21401 non-null int64
family_status       21401 non-null object
family_status_id    21401 non-null int64
gender              21401 non-null object
income_type         21401 non-null object
debt                21401 non-null int64
total_income        21401 non-null int64
purpose             21401 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.1+ MB


### Вывод

Заменили вещественный тип данных на целочисленный

### Обработка дубликатов

In [37]:
# Посмотрим на дубликаты
df[df.duplicated()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,NaN,41,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для семьи
4182,1,NaN,34,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,142594,свадьба
4851,0,NaN,60,среднее,1,гражданский брак,1,F,пенсионер,0,118514,свадьба
5557,0,NaN,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
7808,0,NaN,57,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы
8583,0,NaN,58,высшее,0,Не женат / не замужем,4,F,пенсионер,0,118514,дополнительное образование
9238,2,NaN,34,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для сдачи
9528,0,NaN,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,118514,операции со своей недвижимостью
9627,0,NaN,56,среднее,1,женат / замужем,0,F,пенсионер,0,118514,операции со своей недвижимостью
10462,0,NaN,62,среднее,1,женат / замужем,0,F,пенсионер,0,118514,покупка коммерческой недвижимости


### Вывод

Можно заметить, что все дубликаты появились из-за того, что были пропущены данные о стаже и доходе. Скорее всего, это разные люди, у которых совпали остальные характеристики. Удалять дубликаты не будем.

### Лемматизация

In [38]:
# Посмотрим еще раз на цели клиентов
df.purpose.value_counts()

свадьба                                   796
на проведение свадьбы                     772
сыграть свадьбу                           769
операции с недвижимостью                  673
покупка коммерческой недвижимости         661
покупка жилья для сдачи                   651
операции с жильем                         648
операции с коммерческой недвижимостью     646
жилье                                     642
покупка жилья                             641
покупка жилья для семьи                   640
недвижимость                              632
строительство собственной недвижимости    628
операции со своей недвижимостью           626
строительство жилой недвижимости          622
строительство недвижимости                620
покупка своего жилья                      619
покупка недвижимости                      618
ремонт жилью                              609
покупка жилой недвижимости                603
на покупку своего автомобиля              504
заняться высшим образованием      

В целях клиентов четко выделяются четыре группы:
- свадьба
- жилье/недвижимость
- авто
- образование

Создадим новый столбец с этими категориями

In [39]:
# Посмотрим, не упустили ли мы большую группу целей
from collections import Counter

lemmas = Mystem().lemmatize(' '.join(df.purpose))
lemmas

Counter(lemmas).most_common()[:20]

[(' ', 54884),
 ('недвижимость', 6329),
 ('покупка', 5879),
 ('жилье', 4450),
 ('автомобиль', 4288),
 ('образование', 3997),
 ('с', 2906),
 ('операция', 2593),
 ('свадьба', 2337),
 ('свой', 2224),
 ('на', 2218),
 ('строительство', 1870),
 ('высокий', 1368),
 ('получение', 1311),
 ('коммерческий', 1307),
 ('для', 1291),
 ('жилой', 1225),
 ('сделка', 939),
 ('заниматься', 908),
 ('дополнительный', 902)]

In [19]:
%%time
def purpose_to_category(purpose):
    m = Mystem()
    lemmas = m.lemmatize(purpose)
    if 'свадьба' in lemmas:
        return 'свадьба'
    elif 'жилье' in lemmas or 'недвижимость' in lemmas:
        return 'жилье/недвижимость'
    elif 'автомобиль' in lemmas:
        return 'авто'
    elif 'образование' in lemmas:
        return 'образование'
    else: 
        return 'другое'


unique_purposes = df.purpose.unique()
purposes_dict ={}

for purpose in unique_purposes:
    purposes_dict[purpose] = purpose_to_category(purpose)

CPU times: user 39.1 ms, sys: 214 ms, total: 253 ms
Wall time: 36.1 s


In [25]:
df['purpose_cat'] = df.purpose.replace(purposes_dict)
df.purpose_cat.value_counts()

жилье/недвижимость    10779
авто                   4288
образование            3997
свадьба                2337
Name: purpose_cat, dtype: int64

### Вывод

Все цели объеденены в группы. Теперь можно удалить лишние столбцы и провести категоризацию данных.

### Категоризация данных

In [26]:
# Проведем категоризацию образования клиентов
education_dict = {0 : 'высшее',
                  1 : 'среднее',
                  2 : 'неоконченное высшее',
                  3 : 'начальное',
                  4 : 'ученая степень'}

df.education = df.education_id.replace(education_dict)

# Приведем оставшиеся колонки к типу category
category_cols = ['education', 'family_status', 'gender', 'income_type', 'purpose_cat']
df[category_cols] = df[category_cols].astype('category')

# Удалим старые столбцы
df.drop(axis=1, columns=['education_id', 'family_status_id', 'purpose'], inplace=True)

In [27]:
# Проведем категоризацию доходов клиентов
def income_cat(income):
    if income < 70000:
        return '<70k'
    elif income < 110000:
        return '70k-110k'
    elif income < 160000:
        return '110k-150k'
    elif income < 200000:
        return '150k-200k'
    elif income < 300000:
        return '200k-300k'
    else:
        return '>300k'
    
df['total_income_cat'] = df.total_income.apply(income_cat).astype('category')
df.total_income_cat = df['total_income_cat'].cat.set_categories(['<70k', '70k-110k', '110k-150k',
                                                                 '150k-200k', '200k-300k', '>300k'],
                                                                ordered=True
                                                               )

In [28]:
df.head()

,children,days_employed,dob_years,education,family_status,gender,income_type,debt,total_income,purpose_cat,total_income_cat
0,1,8437.673028,42,высшее,женат / замужем,F,сотрудник,0,253875,жилье/недвижимость,200k-300k
1,1,4024.803754,36,среднее,женат / замужем,F,сотрудник,0,112080,авто,110k-150k
2,0,5623.422610,33,среднее,женат / замужем,M,сотрудник,0,145885,жилье/недвижимость,110k-150k
3,3,4124.747207,32,среднее,женат / замужем,M,сотрудник,0,267628,образование,200k-300k
4,0,NaN,53,среднее,гражданский брак,F,пенсионер,0,158616,свадьба,110k-150k


### Вывод

Выделили категории доходов клиентов в колонке `total_income_cat`. Привели колонки `education`, `family_status`, `gender`, `income_type`, `purpose_cat`, `total_income_cat` к типу данных *category*. Теперь данные готовы для ответов на поставленные вопросы.

### Шаг 3. Ответьте на вопросы

#### Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [29]:
df.children.value_counts()

0    14148
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

In [30]:
df1 = df[df.children == 0] # нет детей
df2 = df[df.children > 0] # есть дети

print(f'Доля просроченных кредитов при отсутсвии детей: {df1.debt.mean():.1%}')
print(f'Доля просроченных кредитов при наличии детей: {df2.debt.mean():.1%}')

Доля просроченных кредитов при отсутсвии детей: 7.5%
Доля просроченных кредитов при наличии детей: 9.2%


In [31]:
# Доля просроченных кредитов с учетом количества детей
df.pivot_table(index=['children'], 
                            values='debt',
                            aggfunc='mean'
                           ).round(4) * 100

,debt
children,
0,7.51
1,9.22
2,9.44
3,8.18
4,9.76
5,0.00


### Вывод

Зависимость между наличием детей и возвратом кредита в срок есть.
Клиенты с детьми немного чаще просрочивают выплаты по кредитам. Видимо, связано это с большей финансовой нагрузкой.

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [32]:
df.family_status.value_counts()

женат / замужем          12302
гражданский брак          4159
Не женат / не замужем     2799
в разводе                 1189
вдовец / вдова             952
Name: family_status, dtype: int64

In [33]:
# Посмотрим на процент просроченных выплат
df.pivot_table(index=['family_status'], 
               values='debt',
               aggfunc='mean'
              ).sort_values('debt').round(4)*100

,debt
family_status,
вдовец / вдова,6.62
в разводе,7.06
женат / замужем,7.54
гражданский брак,9.26
Не женат / не замужем,9.75


### Вывод

Однозначно можно сделать вывод, что у клиентов состоящих в браке доля просроченных кредитов меньше, чем не состоящих в браке или проживющих в гражданском браке. 

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [34]:
df.total_income_cat.value_counts()

110k-150k    7105
70k-110k     4153
150k-200k    3641
200k-300k    3564
>300k        1474
<70k         1464
Name: total_income_cat, dtype: int64

In [35]:
# Доля просроченных кредитов с учетом уровня дохода
df.pivot_table(index=['total_income_cat'], 
                            values='debt',
                            aggfunc='mean'
              ).sort_values(by='debt').round(4)*100

,debt
total_income_cat,
<70k,6.90
200k-300k,7.01
>300k,7.19
150k-200k,8.46
70k-110k,8.52
110k-150k,8.63


### Вывод

Чаще всего просрочивают кредиты клиенты с зарплатой 70-200 т.р. Реже всего - клиенты с зарплатой меньше 70 т.р. и кленты с заралатой больше 200 т.р.

- в первом случае, возможно, это связано с меньшими размерами выплат выданных кредитов, т.к. маленькую сумму легче заработать или перезанять
- во втором случае, видимо, влияет более ответственный подход клиентов и работников банка

#### Как разные цели кредита влияют на его возврат в срок?

In [36]:
df.purpose_cat.value_counts()

жилье/недвижимость    10779
авто                   4288
образование            3997
свадьба                2337
Name: purpose_cat, dtype: int64

In [37]:
df.pivot_table(index=['purpose_cat'], 
                            values='debt',
                            aggfunc='mean'
              ).sort_values(by='debt').round(4)*100

,debt
purpose_cat,
жилье/недвижимость,7.24
свадьба,7.83
образование,9.23
авто,9.33


### Вывод

Клиенты чаще всего просрочивают выплаты кредитов на образование и авто. Реже -  на нежвижимость и свадьбу.  

### Общая таблица 

In [38]:
df.pivot_table(index=['purpose_cat', 'family_status'], 
                            columns=['total_income_cat'],
                            values='debt',
                            aggfunc=['mean']
              ).round(4)*100

mean                               \
total_income_cat                           <70k 70k-110k 110k-150k 150k-200k   
purpose_cat        family_status                                               
авто               Не женат / не замужем   9.09    10.48     13.42     18.10   
                   в разводе              11.76     8.33      7.22     11.36   
                   вдовец / вдова          8.82     7.69     11.39      8.70   
                   гражданский брак        9.38    14.29     12.74     13.11   
                   женат / замужем         8.29    10.20      7.56      7.89   
жилье/недвижимость Не женат / не замужем   7.07     8.53      6.49     11.19   
                   в разводе               2.78     7.03      9.86      3.91   
                   вдовец / вдова          1.79     1.64      6.45     10.13   
                   гражданский брак        1.79     8.15     10.94     11.35   
                   женат / замужем         6.79     7.29      7.49      6.40   
образование        Не женат / не замужем  11.54    10.37     11.86     10.00   
                   в разводе               6.67     9.09      2.50      9.30   
                   вдовец / вдова          8.70    10.00      6.49      7.41   
                   гражданский брак        3.33    18.99     17.83     15.49   
                   женат / замужем         5.79     8.84      9.43      8.90   
свадьба            гражданский брак        9.33     8.44      9.17      7.59   

                                                           
total_income_cat                         200k-300k  >300k  
purpose_cat        family_status                           
авто               Не женат / не замужем     11.76   9.76  
                   в разводе                  3.85   4.76  
                   вдовец / вдова             8.70   0.00  
                   гражданский брак          10.67   3.57  
                   женат / замужем            7.84   8.87  
жилье/недвижимость Не женат / не замужем      8.03   8.77  
                   в разводе                  5.36   7.27  
                   вдовец / вдова             4.29   8.70  
                   гражданский брак           5.36  13.43  
                   женат / замужем            6.46   6.21  
образование        Не женат / не замужем     11.61   3.85  
                   в разводе                 16.13   0.00  
                   вдовец / вдова             6.90   0.00  
                   гражданский брак           8.96  14.81  
                   женат / замужем            6.90   6.11  
свадьба            гражданский брак           4.81   5.96

### Шаг 4. Общий вывод

На возврат кредита в срок влияют все рассмотренные нами критерии: наличие детей, семейное положение, уровень доходов и цель. Их необходимо учитывать при построении модели кредитного скоринга.

Мы обработали и проанализировали предоставленные данные о платежеспособности клиентов.
Выяснили, что на возврат кредита в срок влияют все рассмотренные нами критерии: 
- наличие детей: просроченных кредитов у клиентов с детьми почти на 2% больше, чем у клиентов без детей, от количества детей данные зависят слабо
- семейное положение: больше всех просрочивают кредиты клиенты в гражданском браке и не женатые/не замужем, в отличии от женатых, вдовцов и разведенных - разница около 2.5%
- уровень доходов: клиенты со средней зарпалтой (70-200 т.р.) просрочивают кредиты чаще остальных на 1.5%
- цель: кредиты на жилье и свадьбы просрочивают реже чем на автомобили и образование  в среднем на 1.5%   
    
Все эти данные о клиентах необходимо учитывать при построении модели кредитного скоринга.

В дальнейшей работе можно так же проанализировать данные о стаже, возрасте, типе занятости и образовании клиентов.